In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt

import multirtd.params as params
from multirtd.dubins_planner import DubinsPlanner

%load_ext autoreload
%autoreload 2

Test scenario with obstacles

In [ ]:
p_0 = np.array([[-5], [0]])

planner = DubinsPlanner()
# Set planner goal and obstacles
planner.p_goal = np.array([[5], [0]])
# (center, radius) tuples
planner.obstacles = [(np.array([3, 1]), 1.5),
                     (np.array([0, -1.5]), 1.5),
                     (np.array([-3, 1]), 1.5)]

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

#ax.scatter(p_0[0], p_0[1], s=50, marker='^', color='b')
ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

plt.show()

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
x = np.zeros(3)
# State history
X = []
planning_times = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan(x)
        planning_times.append(time.time() - start_t)
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    x = trajectory[traj_idx]
    X.append(x)
    traj_idx += 1

    if np.linalg.norm(x[:-1] - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

# P = np.hstack(P)
print("Average planning time: ", np.mean(planning_times), " seconds")

Test scenario with multiple obstacles and uncertainty regions